## 카카오 로컬 API 활용
- 도로명 주소 --> 위도, 경도 좌표

In [104]:
import requests, json
import pandas as pd
from urllib.parse import quote
from tqdm import tqdm

- API Key

In [105]:
with open('keys/카카오apiKey.txt') as file:
    kakao_key = file.read()

- URL 만들기

In [106]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울특별시 은평구 진흥로5길 4 (역촌동)'
url = f'{base_url}?query={quote(addr)}'

In [107]:
# Header: Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {kakao_key}'}

- Kakao Local API 호출하여 결과 가져오기

In [108]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 은평구 역촌동 28-15',
    'b_code': '1138010800',
    'h_code': '1138062500',
    'main_address_no': '28',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '은평구',
    'region_3depth_h_name': '역촌동',
    'region_3depth_name': '역촌동',
    'sub_address_no': '15',
    'x': '126.920641600837',
    'y': '37.60291486121'},
   'address_name': '서울 은평구 진흥로5길 4',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 은평구 진흥로5길 4',
    'building_name': '',
    'main_building_no': '4',
    'region_1depth_name': '서울',
    'region_2depth_name': '은평구',
    'region_3depth_name': '역촌동',
    'road_name': '진흥로5길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.920641600837',
    'y': '37.60291486121',
    'zone_no': '03405'},
   'x': '126.920641600837',
   'y': '37.60291486121'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [109]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [110]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat, lng

(37.60291486121, 126.920641600837)

- 영등포소재 지하철역 정보 완성하기

In [111]:
df = pd.read_csv('data/서울부동산.csv')
df

,ADDRESS,RA_REGNO,RDEALER_NM,CMP_NM,TELNO
0,서울특별시 은평구 진흥로5길 4,다92340000-3128,구범,신화공인중개사사무소,359-1216
1,서울특별시 은평구 진흥로9길 8-18,다92340000-3132,김영희,예스공인중개사사무소,386-5566
2,서울특별시 은평구 불광로22길 28-13,다92340000-3517,최순이,대용공인중개사사무소,02-385-8760
3,서울특별시 은평구 진흥로1길 54,다92340000-3531,임진영,대지공인중개사사무소,02-355-4401
4,서울특별시 은평구 진흥로1길 24,다92340000-3533,서용수,현대공인중개사사무소,382-5050
...,...,...,...,...,...
26603,서울특별시 마포구 독막로 84,92380000-2936,구은숙,롯데부동산공인중개사사무소,325-0053
26604,서울특별시 마포구 월드컵북로 233,92380000-2945,오근오,성산부동산공인중개사사무소,305-5449
26605,서울특별시 마포구 토정로31길 57,92380000-01071,손유아,부동산씨티공인중개사사무소,703-5011
26606,서울특별시 마포구 대흥로 186-1,92380000-01006,윤용철,신흥공인중개사사무소,02-704-3579


In [112]:
df.drop(columns=['RDEALER_NM', 'TELNO'], inplace=True)
df

,ADDRESS,RA_REGNO,CMP_NM
0,서울특별시 은평구 진흥로5길 4,다92340000-3128,신화공인중개사사무소
1,서울특별시 은평구 진흥로9길 8-18,다92340000-3132,예스공인중개사사무소
2,서울특별시 은평구 불광로22길 28-13,다92340000-3517,대용공인중개사사무소
3,서울특별시 은평구 진흥로1길 54,다92340000-3531,대지공인중개사사무소
4,서울특별시 은평구 진흥로1길 24,다92340000-3533,현대공인중개사사무소
...,...,...,...
26603,서울특별시 마포구 독막로 84,92380000-2936,롯데부동산공인중개사사무소
26604,서울특별시 마포구 월드컵북로 233,92380000-2945,성산부동산공인중개사사무소
26605,서울특별시 마포구 토정로31길 57,92380000-01071,부동산씨티공인중개사사무소
26606,서울특별시 마포구 대흥로 186-1,92380000-01006,신흥공인중개사사무소


In [113]:
lat_list, lng_list = [], []
for i in tqdm(df.index):
    url = f'{base_url}?query={quote(df["ADDRESS"][i])}'
    result = requests.get(url, headers=header).json()   
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))
    

  0%|          | 6/26608 [00:00<25:11, 17.59it/s]

 64%|██████▍   | 17042/26608 [16:55<09:30, 16.78it/s]


KeyboardInterrupt: 

In [ ]:
df['위도'] = lat_list
df['경도'] = lng_list
df

In [ ]:
df.to_csv('data/영등포소재 지하철역2.csv', index=False)
pd.read_csv('data/영등포소재 지하철역2.csv')

,이름,주소,위도,경도
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가),37.516463,126.906659
1,신도림역,서울특별시 구로구 경인로 625 (신도림동),37.507084,126.884719
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가),37.534957,126.902731
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동),37.521842,126.924485
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가),37.525700,126.896627
